# Excercise_3 : K-NN on Amazon Fine Food Review Dataset.

****Objective: ****
  To find accuracy of the K-NN model using 10-fold cross_validation(brute,kd_tree) on the amazon_fine_food_review dataset.
  
  **Steps:**
  1. Imprort libraries and datset.
  2. Perform datacleaning , Text preprocessing(removing html-tags,special_characters,convert words to lower case, remove stopwords,remove words len < 2,stemming(Snowball)).
  3. Apply text-to-vector techniques like Bag-of-words,Tf-idf,Average Word2Vec, Tfidf weighted Word2Vec.
  4. For Each Technique find the best accuracy using 10-fold cross_validation with brute and kdtree algorithms.

In [0]:
!pip install gensim

In [0]:
# import libraries
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import sqlite3
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from google.colab import drive

In [0]:
drive.mount('/content/drive')

# Import Data

In [0]:
# using the SQLite Table to read data.
con = sqlite3.connect('/content/drive/My Drive/Colab Notebooks/database.sqlite') 

#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 """, con,coerce_float=True) 


# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 0
    return 1

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

# DataCleaning

In [0]:
#Sorting data according to ProductId in ascending order,default_axis = 0,na_position-default-last.
sorted_data = filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

# Text-Preprocessing.
1. Remove html-tags.
2. Remove punctuations and special characters.
3. remove non-alpha numeric charcters and words with length less than 2.
4. Convert words to lowercase.
5. Remove stopwords.
6. Apply stemming(Snowball Stemming)

In [0]:
stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer
#function to clean the word of any html-tags
def cleanhtml(sentence): 
    cleanr = re.compile('<.*?>') #compiles a pattern.
    cleantext = re.sub(cleanr, ' ', sentence) #replaces cleanr with '' in sentence.
    return cleantext
#function to clean the word of any punctuation or special characters
def cleanpunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
  
if not os.path.isfile('/content/drive/My Drive/Colab Notebooks/final.sqlite'):
    i=0
    str1=' '
    final_string=[]
    s=''
    for sent in tqdm(final['Text'].values):
        filtered_sentence=[]
        sent=cleanhtml(sent) # remove HTMl tags
        for w in sent.split():
            for cleaned_words in cleanpunc(w).split():
                if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                    if(cleaned_words.lower() not in stop):
                        s=(sno.stem(cleaned_words.lower())).encode('utf8')
                        filtered_sentence.append(s)
                    else:
                        continue
                else:
                    continue 
        str1 = b" ".join(filtered_sentence) #final string of cleaned words
        final_string.append(str1)
        i+=1

    final['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 
    final['CleanedText']=final['CleanedText'].str.decode("utf-8")
        # store final table into an SQlLite table for future.
    conn = sqlite3.connect('/content/drive/My Drive/Colab Notebooks/final.sqlite')
    c=conn.cursor()
    conn.text_factory = str
    final.to_sql('Reviews', conn,  schema=None, if_exists='replace',index=True, index_label=None, chunksize=None, dtype=None)
    conn.close()

In [0]:
# importing the preprocessed dataset

con = sqlite3.connect('/content/drive/My Drive/Colab Notebooks/final.sqlite')
final = pd.read_sql_query("""SELECT * FROM Reviews WHERE Score != 3""",con)
#random sampling 

#Sort the datset by timstamp.
final.sort_values(by=['Time'],ascending=True,inplace=True,na_position='first')
df = final[0:100000]
df['Score'].value_counts()

1    87729
0    12271
Name: Score, dtype: int64

**Observations:** 
1. Imbalanced sample.

**Function to apply K-NN 10-fold cross validation on the dataset using brute-force and kd-tree algorithms**

In [0]:
#Function to apply 10-fold cross_validation using brute-force algorithm
def brute_knn(x,y):
    
    cv_scores=[]
    listo = list(range(0,20))
    neighbors = list(filter(lambda x : x % 2 != 0,listo))
    
    for k in neighbors:
    
        knn = KNeighborsClassifier(n_neighbors=k,algorithm='brute',weights='distance')


    optimal_k = neighbors[MSE.index(min(MSE))]
    print('The optimal k value using brute-force is ',optimal_k)
    optimal_knn = KNeighborsClassifier(n_neighbors=optimal_k,algorithm='brute',weights='distance')
    optimal_knn.fit(x_train,y_train)

    pred = optimal_knn.predict(x_test)

    acc = accuracy_score(y_test,pred,normalize=True) * 100.0
    
    return acc

#Function to apply 10-fold cross_validation using kd_tree algorithm

def kdtree_knn(x,y):
    
    listp = list(range(0,20))
    neighbors = list(filter(lambda x : x % 2 != 0,listp))
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)

    for k in neighbors:
    
        knn = KNeighborsClassifier(n_neighbors=k,algorithm='kd_tree',weights='distance')
        cross_val = cross_val_score(knn,x_train,y_train,cv=10)
        cv_scores.append(cross_val.mean())

    MSE = [1 - x for x in cv_scores]

    optimal_k = neighbors[MSE.index(min(MSE))]
    print('The optimal k value using kd_tree is ',optimal_k)
    optimal_knn = KNeighborsClassifier(n_neighbors=optimal_k,algorithm='kd_tree',weights='distance')
    optimal_knn.fit(x_train,y_train)

    pred = optimal_knn.predict(x_test)

    accuracy = accuracy_score(y_test,pred,normalize=True) * 100.0
    
    return accuracy





**Observations:**
1. The dataset is split into 70:30,train and test data.
1. 10-fold cross_validation is done using k values from 1 - 19 for both brute and kd_tree algorithms.
2. Optimal k value is obtained and the knn model is tested on the test sample , 
      and accuracy score is obtained. 


# K-NN -  BOW

In [0]:
bow = CountVectorizer()
bow_matrix = bow.fit_transform(df['CleanedText']) 
standardized_data = StandardScaler(with_mean=False).fit_transform(bow_matrix)
x_1=standardized_data.todense()
y_1=df['Score']

acc_b = brute_knn(x_1,y_1)
acc_k = kdtree_knn(x_1,y_1)

print('The accuracy using brute-force is ',acc_b)
print('The accuracy using kd_tree is ',acc_k)

The optimal k value using brute-force is  3
The optimal k value using kd_tree is  3
The accuracy using brute-force is  85.33333333333334
The accuracy using kd_tree is  85.33333333333334


**Observations:**
1. The optimal k value for both brute and kd_tree algorithms for this sample is 3.
1. The accuracies of the model using both the brute-force and kd_tree algorithms for this sample are same.

# K-NN - TF-IDF

In [0]:
tfidf_model = TfidfVectorizer(ngram_range=(1,2))
tfidf_matrix = tfidf_model.fit_transform(df['CleanedText'])
standardized_data = StandardScaler(with_mean=False).fit_transform(tfidf_matrix)
x_2 = standardized_data.todense()
y_2 = df['Score']

acc_b = brute_knn(x_2,y_2)
acc_k = kdtree_knn(x_2,y_2)
print('The accuracy using brute-force is ',acc_b)
print('The accuracy using kd_tree is ',acc_k)

The optimal k value using brute-force is  1
The optimal k value using kd_tree is  1
The accuracy using brute-force is  85.0
The accuracy using kd_tree is  85.0


**Observations:**
1.  The optimal k value for both brute-force and kd_tree is 1.
2. The accuracies for both the algorithms for the sample is same.

# K-NN - AverageWord2Vec

In [0]:
#Average Word2Vector.
list_of_sent=[]  #list of sentences of the cleanedtext.
for sent in df['CleanedText'].values:
    list_of_sent.append(sent.split())
    
w2v_model = Word2Vec(list_of_sent,min_count=5,size=50)
w2v_words = list(w2v_model.wv.vocab)

final_vec = [] #final list of vectors.

for sent in list_of_sent:
    sent_vec = np.zeros(50)
    count = 0
    for word in sent:
        if word in w2v_words:
            vec = w2v_model[word]
            sent_vec += vec
            count += 1
    if count != 0:
        sent_vec / count
        
    final_vec.append(sent_vec)

standardized_data = StandardScaler().fit_transform(final_vec)
x_3 = standardized_data
y_3 = df['Score']

acc_b = brute_knn(x_3,y_3)
acc_k = kdtree_knn(x_3,y_3)

print('The accuracy of the model using brute-force is',acc_b)
print('The accuracy using kd_tree is ',acc_k)


The optimal k value using brute-force is  19
The optimal k value using kd_tree is  19
The accuracy of the model using brute-force is 84.66666666666667
The accuracy using kd_tree is  84.66666666666667


**Observations:**
1. The accuracies using both brute-force and kd_tree algorithms are same.


# K-NN - TF-IDFweightedWord2Vec

In [0]:
#Tfidf model.
tfidf_model = TfidfVectorizer()
tfidf_matrix = tfidf_model.fit_transform(df['CleanedText'])

#create dictionary of idf of words.
idf_values = dict(zip(tfidf_model.get_feature_names(),list(tfidf_model.idf_)))

w2v_tfidf_vec = [] #final list of vectors.

for sent in list_of_sent:
    total_tfidf = 0
    for word in sent:
        if word in w2v_words:
            vec = w2v_model[word]
            tfidf = idf_values[word] * (sent.count(word)/ len(sent))
            total_tfidf += tfidf
            
            sent_vec += vec * tfidf
            
    if total_tfidf != 0:
        sent_vec / total_tfidf
    w2v_tfidf_vec.append(sent_vec)
    
x_4 = StandardScaler().fit_transform(w2v_tfidf_vec)
y_4 = df['Score']

acc_b = brute_knn(x_4,y_4)
acc_k = kdtree_knn(x_4,y_4)
print(acc_b)
print(acc_k)

The optimal k value using brute-force is  5
The optimal k value using kd_tree is  3
85.0
85.0


**Observations:**
1. The accuracies for both the algorithms are same.

# Conclusion:
1. The 2000 samples taken was imbalanced, like the original dataset(35k,15k).
1. The accuracy values for both brute-force and kd_tree algorithms are same for the  sample taken .
